# TSA Chapter 4: Case Study - SARIMA Forecast

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch4/TSA_ch4_case_forecast/TSA_ch4_case_forecast.ipynb)

Out-of-sample SARIMA forecast for airline passengers.

In [ ]:
!pip install numpy pandas matplotlib statsmodels -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import acf, pacf
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Load airline passengers
data = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv',
                    parse_dates=['Month'], index_col='Month')
passengers = data['Passengers']
passengers.index.freq = 'MS'
log_passengers = np.log(passengers)

In [ ]:
train = log_passengers[:-24]; test = log_passengers[-24:]
model = SARIMAX(train, order=(0,1,1), seasonal_order=(0,1,1,12), enforce_stationarity=False, enforce_invertibility=False)
fit = model.fit(disp=False); fcast = fit.get_forecast(steps=24); ci = fcast.conf_int()
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(train.index[-48:], np.exp(train.values[-48:]), color=BLUE, lw=1.5, label='Training')
ax.plot(test.index, np.exp(test.values), color=GREEN, lw=2, label='Actual')
ax.plot(test.index, np.exp(fcast.predicted_mean.values), color=RED, lw=2, ls='--', label='Forecast')
ax.fill_between(test.index, np.exp(ci.iloc[:,0]), np.exp(ci.iloc[:,1]), color=RED, alpha=0.15, label='95% CI')
ax.set_title('SARIMA$(0,1,1)(0,1,1)_{12}$ Forecast', fontweight='bold')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=4, frameon=False)
plt.tight_layout(); save_chart(fig, 'ch4_case_forecast'); plt.show()